# annotate family relations
* train a model for family relation types

In [ ]:
import ast
import random
from spacytei.train import batch_train

In [ ]:
persons = Person.objects.filter(is_related_person__isnull=False)
persons.count()

In [ ]:
items = [x.written_name for x in persons]
items = sorted(iter(items), key=lambda k: random.random())
filename = "person__related.txt"
with open(filename, 'w', encoding="utf-8") as f:
    for x in items:
        f.write("{}".format(x) + '\n')

In [ ]:
file = "./training_data/invs_fam.jsonl"

In [ ]:
with open(file) as f:
    TRAIN_DATA = f.readlines()
train_data = [ast.literal_eval(x) for x in TRAIN_DATA]
print(len(train_data))
train_data[4]

In [ ]:
batch_train(train_data=train_data, output_dir='./data/main_fam', new_label='FAM', eval_split=0.2, n_iter=8)

In [ ]:
samples = [
    "Maria Hueberin (Tochter des Michael Hueber und der Ursula Mayrin; Nichte des Verstorbenen)",
    "Veit [Vitus] Schiferegger [Schifferegger] (Sohn des Verstorbenen)",
    "Johann Kofler [Hans Kofler/Joseph Kofler?] (Tischlermeister), Bruneck, (Kurator des Andre Papprian, des Johann Papprian [Hans Papprian], der Maria Papprianin und der Magdalena Papprianin)",
    "Maria Göllerin (abwesende Tochter der Maria Aichnerin, einer Halbschwester des Verstorbenen)",
]

In [ ]:
import spacy

In [ ]:
nlp = spacy.load(r"./data/main_fam")

In [ ]:
fam_scheme, _ = SkosCollection.objects.get_or_create(name="Familiäre Beziehungen")

In [ ]:
for x in persons:
    doc = nlp(x.written_name)
    ents = doc.ents
    if len(ents) == 1:
        fam_rel_type = None
        perper = None
        try:
            inv = x.is_related_person.all()[0]
        except IndexError:
            inv = None
        try:
            main_person = inv.main_person.all()[0]
        except IndexError:
            main_person = None
        if main_person is not None:
            fam_rel_type, _ = SkosConcept.objects.get_or_create(pref_label="{}".format(ents[0]))
            fam_rel_type.collection.add(fam_scheme)
            perper, _ = PersonPerson.objects.get_or_create(
                source=x,
                target=main_person,
                rel_type=fam_rel_type
            )
            perper.save()
    else:
        pass

In [ ]:
# PersonPerson.objects.all().count()

In [ ]:
# PersonPerson.objects.all().delete()